In [2]:
import numpy as np
from torchvision import transforms
from PIL import Image
import tritonclient.http as httpclient

def rn50_preprocess(img_path="./img1.jpg"):
    img = Image.open(img_path)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return preprocess(img).numpy()

transformed_img = np.expand_dims(rn50_preprocess(), axis=0)

In [1]:
# Setup a connection with the Triton Inference Server.
triton_client = httpclient.InferenceServerClient(url="triton-classifier-route-triton-image-deployments.apps.nebula.sl")

# Specify the names of the input and output layer(s) of our model.
test_input = httpclient.InferInput("image", transformed_img.shape, datatype="FP32")
test_input.set_data_from_numpy(transformed_img, binary_data=True)

test_output = httpclient.InferRequestedOutput("final_output", binary_data=False)

# Querying the server
results = triton_client.infer(model_name="ensemble-model", inputs=[test_input], outputs=[test_output])
test_output_fin = results.as_numpy('final_output')

print(test_output_fin)

NameError: name 'httpclient' is not defined